## sentence transformer setting

In [ ]:
pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 6.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## google colab setting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=(SettingWithCopyWarning))

In [ ]:
# data path
path = '/content/drive/MyDrive/python/공공데이터 공모전/data/'

## 1. DSC(import) embedding

In [ ]:
#LLM의 수입 품목 데이터
data = pd.read_json(path+'LLMimport.json')

### preprocessing

In [ ]:
# LLM result preprocessing
def preprocess(x):
  if ':' in x :
    x = x[x.find(':')+1:]
  # ":"가 있으면 그 전까지 문장 제거, '\n*' 는 ','으로 replace, "" 큰따옴표 제거
  x = x.replace('\n*', ',').replace('\n, ', '').replace('"', '').replace('\n', ',').strip()
  if x[:2] == ',,':
    x = x[2:]
  if ',,' in x:
    x = x.split(',,')[0] #상단 코드 \n replace 때문에 생긴 패턴으로, 뒷부분이 부연설명이므로 삭제
  # 문장 구분자 : 온점 붙이기
  if x[-1] != '.':
    x = x+'.'
  x = x.replace('...', '').replace('* ', '').replace(';', ', ').replace(',', ', ').replace('  ', ' ').strip()
  return x

#적용
data['LLMimport'] = data['LLMimport'].apply(lambda x : preprocess(x))

In [ ]:
# 문장 형태로 기존의 DSC에 결합
data['DSC(import)'] = data['DSC'] + '. This company may need ' + data['LLMimport']

In [ ]:
# ex (1)
data['DSC(import)'][32]

'surgical and medical instruments  manufacturing medical, surgical, ophthalmic, and veterinary instruments and apparatus. medical equipment and supplies surgical and medical instruments surgical and medical instrument manufacturing manufacture of medical and dental instruments and supplies manufacture of medical and dental instruments and supplies manufacture of medical and dental instruments and suppliesmedical and surgical equipment manufacturing. This company may need Stainless steel (for manufacturing surgical instruments), Titanium alloy (for manufacturing implants or other surgical devices), Silicone (for manufacturing implantable devices or prosthetics), Polypropylene (for manufacturing medical device components such as filters or tubing), Ceramic (for manufacturing dental implants or orthopedic joint replacements), Rubber (for manufacturing medical device components such as gaskets or seals), Teflon (for manufacturing surgical instruments or medical device components that requi

In [ ]:
# ex (2)
data['DSC(import)'][10]

'fresh fruits and vegetables  the wholesale distribution of fresh fruits and vegetables. grocery wholesale fruits, fresh fresh fruit and vegetable merchant wholesalers wholesale of fruit and vegetables wholesale of food, beverages and tobacco wholesale of fruit and vegetablesfruit and vegetable wholesaling. This company may need Containers, crates, pallets, refrigeration equipment, trucks, shelving units, scales, packaging materials (boxes, bags, etc.), produce bags, cold storage facilities, warehouse management software, inventory tracking devices, temperature monitoring devices, and forklifts.'

### embedding

In [ ]:
data['transformer_embedding'] = '0' #object로 해줘야

In [ ]:
for i in range(len(data)):
  text = data['DSC(import)'][i]
  try:
    vec = list(model.encode(text))
    data['transformer_embedding'][i] = vec
  except:
    pass

In [ ]:
data.to_json(path+'data(import)_transformer_embedding.json')

In [ ]:
data

,DSC,LLMimport,transformer_embedding
0,"automotive repair shops, nec specialized auto...","Brake linings, carburetors, radiators, fuel fi...","[0.005848373, -0.01318487, -0.005108098, 0.011..."
1,"general farms, primarily animals, nsk derives...","Feeding equipment, veterinary supplies, and fa...","[0.06567818, 0.0379473, -0.016570069, -0.01420..."
2,fish and seafoods the wholesale distribution ...,"Cooler units, pallets, crates, boxes, gloves, ...","[0.041967563, -0.0011303811, -0.03717378, -0.0..."
3,"new and used car dealers, nsk manufactures a ...","Steel alloys for car chassis and body panels, ...","[0.0065446342, -0.011539101, -0.014426424, 0.0..."
4,"automotive stampings, nsk manufacturing autom...","Steel sheets, Aluminum alloys, Copper alloys, ...","[0.019126616, -0.081012644, -0.0019861253, 0.0..."
...,...,...,...
1713,"printing and writing paper, nsk the wholesale...","Ink, toners, and printing supplies.","[0.07137083, -0.006677211, 0.016515607, 0.0001..."
1714,leather and sheep-lined clothing manufacturin...,"Leather hides, Sheep pelts or wool, Thread (sp...","[-0.03331482, -0.012678897, 0.012401181, 0.020..."
1715,"electronic parts and equipment, nec, nsk the ...","Integrated circuits, Microprocessors, Printed ...","[0.03562802, -0.06995159, -0.0032274586, -0.02..."
1716,"bottled and canned soft drinks, nsk manufactu...","Glass or plastic bottles, Aluminum cans, Label...","[0.06694527, -0.02107171, -0.023391789, -0.033..."


## 2. HS10 전처리 embedding

In [ ]:
# 수작업으로 전처리한 HS10 데이터
hs = pd.read_json(path+'HS10전처리.json')

### embedding

In [ ]:
hs['transformer_embedding'] = '0'

In [ ]:
for i in range(len(hs)):
  text = hs['품목명'][i]
  try:
    vec = list(model.encode(text))
    hs['transformer_embedding'][i] = vec
  except:
    pass

In [ ]:
hs.to_json(path+'hs_transformer_embedding.json')

## 3. DSC 내의 except & other than embedding

In [ ]:
data = pd.read_excel(path+'비식별된 해외기업별 영문 텍스트데이터.xlsx')

### preprocessing

In [ ]:
# DSC 내에 except 토큰이 있는 경우만 필터링
def if_except(text):
  if 'except' in text:
    return 1
  else:
    return 0
data['except'] = data['DSC'].apply(lambda x : if_except(x))

# DSC 내에 other than 토큰이 있는 경우만 필터링
def if_other_than(text):
  if 'other than' in text:
    return 1
  else:
    return 0
data['other_than'] = data['DSC'].apply(lambda x : if_other_than(x))

#적용
data = data[(data['except']==1)|(data['other_than']==1)].drop_duplicates('DSC').reset_index(drop=True)
data = pd.DataFrame(data['DSC'])

In [ ]:
# except에 해당하는 내용만 추출
def except_parentheses(x):
  if 'except' in x:
    x = x.replace(';', '.').replace(',', '.').replace(')', '.')
    x = x[x.find('except'):]
    x = x[:x.find('.')].replace('except ', '')
    return x
  else:
    return '0'

# other than에 해당하는 내용만 추출
def other_parentheses(x):
  if 'other than' in x:
    x = x.replace(';', '.').replace(',', '.').replace(')', '.')
    x = x[x.find('other than'):]
    x = x[:x.find('.')].replace('other than ', '')
    return x
  else:
    return '0'

#적용
data['except'] = data['DSC'].apply(lambda x : except_parentheses(x))
data['other'] = data['DSC'].apply(lambda x : other_parentheses(x))

### embedding

In [ ]:
data['except_embedding'] = '0'
data['other_embedding'] = '0'

In [ ]:
# except embedding
for i in list(data[data['except']!='0'].index):
  ex = data['except'][i]
  vec = list(model.encode(ex))
  data['except_embedding'][i] = vec

In [ ]:
# other than embedding
for i in list(data[data['other']!='0'].index):
  ot = data['other'][i]
  vec = list(model.encode(ot))
  data['other_embedding'][i] = vec

In [ ]:
data.to_json(path+'ex_other_transformer_embedding.json')